In [1]:
##########################
# Do Not Use This Script.
# It Is Not Complete Yet.
##########################

In [1]:
## To download the dataset run the next 4 lines and follow the insturctions:
#! wget http://gvv.mpi-inf.mpg.de/3dhp-dataset/mpi_inf_3dhp.zip
#! unzip mpi_inf_3dhp.zip
#! chmod +x mpi_inf_3dhp/get_dataset.sh
#! chmod +x mpi_inf_3dhp/get_testset.sh

## Edit config file next
## RUN get_dataset.sh next
## RUN get_testset.sh next

##HINT: set ready_to_download=1 comment line 9 and uncomment line 10

In [2]:
import time
import numpy as np
import tensorflow as tf

from tqdm import tqdm

from src.vnect_model import VNect

from src.mpi_inf_3dhp import Mpi_Inf_3dhp

In [4]:

# Missing files for this:
#data = Mpi_Inf_3dhp('./mpi_inf_3dhp')

def dataGenerator():
    i = 0
    while True:
        x = np.random.random((368, 368, 3)) / 255 - 0.4
        y = np.random.random((46, 46, 4*21))
        yield x, y
        i += 1

        
        
## TFRecord seems to be not a good option as it consumes lots of space on the disk
#g = dataGenerator()
#
#def _floats_feature(value):
#    return tf.train.Feature(float_list=tf.train.FloatList(value=list(value.reshape(-1))))
#
#
#
#def _bytes_feature(value):
#    """Returns a bytes_list from a string / byte."""
#    #if isinstance(value, type(tf.constant(0))):
#    value = value.tobytes() # BytesList won't unpack a string from an EagerTensor.
#    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
#
#def serializeData(x,y0, y1, y2, y3):
#    feature = {
#      'img': _bytes_feature(x),
#      'hm': _bytes_feature(y0),
#      'xhm': _bytes_feature(y1),
#      'yhm': _bytes_feature(y2),
#      'zhm':_bytes_feature(y3),
#    }
#
#    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
#    return example_proto.SerializeToString()
#
#with tf.io.TFRecordWriter("./data/example.tfrecords") as writer:
#    for i in range(1000):
#        x, y = next(g)
#        example = serializeData(x, y[0], y[1], y[2], y[3])
#        writer.write(example)

g = dataGenerator()

images = []
hmaps = []

for i in tqdm(list(range(100))):
    x, y = next(g)
    images.append(x)
    hmaps.append(y)

100%|██████████| 100/100 [00:00<00:00, 159.52it/s]


In [6]:
modelClass = VNect()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 368, 368, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 184, 184, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 92, 92, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
res2a_branch2a (Conv2D)         (None, 92, 92, 64)   4160        pool1[0][0]                      
______________________________________________________________________________________________

In [7]:
model = modelClass.getModel()

# loss = tf.losses.mean_squared_error(labels, heatmap, scope='loss') / batch_size
# loss = tf.reduce_sum(tf.nn.l2_loss(tf.concat(labels, axis=0) - tf.concat(heatmap, axis=0)))*2/(46*46*21)/32
# loss = tf.reduce_mean(tf.nn.l2_loss(labels * heatmap))

optimizer = tf.optimizers.Adadelta(0.001, rho=0.95, epsilon=1e-07)
model.compile(optimizer, "MSE")



In [8]:
batch_size = 32
epochs = 10
total_items = 100

dataset = tf.data.Dataset.from_tensor_slices((images, hmaps))
dataset = dataset.repeat(epochs)
dataset = dataset.batch(batch_size)
print(dataset)

num_batches = int(total_items/batch_size)
model.fit(dataset, steps_per_epoch=num_batches, epochs=epochs, verbose=1)

Train for 3 steps
Epoch 1/10
3/3 [==============================] - 9s 3s/step - loss: 0.3331
Epoch 2/10
3/3 [==============================] - 1s 404ms/step - loss: 0.3331
Epoch 3/10
3/3 [==============================] - 1s 393ms/step - loss: 0.3331
Epoch 4/10
3/3 [==============================] - 1s 394ms/step - loss: 0.3331
Epoch 5/10
3/3 [==============================] - 1s 393ms/step - loss: 0.3331
Epoch 6/10
3/3 [==============================] - 1s 405ms/step - loss: 0.3331
Epoch 7/10
3/3 [==============================] - 1s 391ms/step - loss: 0.3331
Epoch 8/10
3/3 [==============================] - 1s 391ms/step - loss: 0.3331
Epoch 9/10
3/3 [==============================] - 2s 503ms/step - loss: 0.3331
Epoch 10/10
3/3 [==============================] - 1s 451ms/step - loss: 0.3331


In [8]:
model.save('models/tf_model/vnect_tf/retrained.h5')